In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import tensorflow as tf
from sklearn import metrics
from sklearn.utils import shuffle
from keras.preprocessing import text
from collections import OrderedDict
import re
import pickle
import time
import sys
import gc
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


## Loaded dataset

In [2]:
def getTrainHoldoutSplit(df, dfLabel, holdoutSize=0.20):
    from sklearn import model_selection 
    dfHoldOut = None
    dfHoldOutLabel = None

    cv_pre = model_selection.StratifiedShuffleSplit(n_splits=1, test_size=holdoutSize, random_state=1)
    for train_index, test_index in cv_pre.split(df, dfLabel):
        y_train, y_test = dfLabel[train_index], dfLabel[test_index]
        x_train, x_test = df.iloc[train_index], df.iloc[test_index]
        df, dfLabel = x_train, y_train
        dfHoldOut, dfHoldOutLabel = x_test, y_test

    print("==================== Data Set ==================================")
    print("Holdout Set => ", dfHoldOut.shape)
    print("Train Set => ", df.shape)
    print("==================== Data Set ==================================")
    return df, dfLabel, dfHoldOut, dfHoldOutLabel

In [3]:
st = time.time()
# df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', nrows=10000)
# dfTest = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=100)
df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
dfTest = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
# df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')

IDENTITY_COLUMNS = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness'
]
AUX_COLUMNS = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
TARGET_COLUMN = 'target'


for column in IDENTITY_COLUMNS:
    df[column] = df[column].apply(lambda x: True if x >= 0.5 else False)

for column in AUX_COLUMNS:
    df[column] = df[column].apply(lambda x: 1 if x >= 0.5 else 0.0)
    
print("Applying sum", time.time() - st)
df['label_imp'] = df[IDENTITY_COLUMNS].sum(axis=1)

print("Calculating class_weight", time.time() - st)
class_weight = np.absolute(df[TARGET_COLUMN].values - np.mean(df[TARGET_COLUMN].values)) + 0.0001
df['class_weight'] = class_weight

max_weight = class_weight.max()
print("IPO", time.time() - st)
# df['class_weight'] = df['class_weight'] + df['label_imp']
# df['class_weight'] = df['class_weight'] .apply(lambda x: max_weight if x > 1 else 1 - x)

df['class_weight'] = df[['class_weight', 'label_imp']].apply(lambda x: max_weight 
                                        if x['label_imp'] >= 1 else x['class_weight'], axis=1)

print("Calculating class_weight_aux", time.time() - st)
class_weight_aux = np.absolute(df[AUX_COLUMNS] - np.mean(df[AUX_COLUMNS], axis=0)) + 0.0001
df['class_weight_aux'] = pd.Series(class_weight_aux.values.tolist())
print("IPO", time.time() - st)

df['class_weight_aux'] = df[['class_weight_aux', 'label_imp']].apply(lambda x: [max_weight] * len(AUX_COLUMNS) if x['label_imp'] >= 1 else x['class_weight_aux'], axis=1)
print("Weights calculated", time.time() - st)
# df = pd.read_csv("../input/fake-news-detection/data.csv")
# df = df.fillna('')
# df['comment_text'] = df['Headline']
# df['pred'] = df['Label']
# dfTest = df
# df['id'] = df.index

def filter_initial_sentence(x):
    return " ".join([re.sub('[^a-z0-9 ]+', '', x) if re.sub('[^a-z0-9 ]+', '', x) else x for x in x.lower().split()])

def tokenize_initial_sentence(x):
    x = re.sub("[’‘'“\"”.\n\t]+", '', x.lower())
    pat = re.compile(r"([—!#$%&()*+,-./:;<=>?@\[\\\]^_`{|}~\t\n…])")
    x = re.sub("[0-9]+", '', x.lower())
    return pat.sub(" \\1 ", x)

df['comment_text'] = df['comment_text'].apply(tokenize_initial_sentence)
dfTest['comment_text'] = dfTest['comment_text'].apply(tokenize_initial_sentence)

df, dfLabel, dfHoldOut, dfHoldOutLabel = getTrainHoldoutSplit(df, df['target'], holdoutSize=0.15)
df, dfLabel, dfVal, dfValLabel = getTrainHoldoutSplit(df, df['target'], holdoutSize=0.18)
# Check Positive vs Negative samples
print("==========Train Sample Distribution =============")
print(dfLabel.value_counts())
print("==========Validation Sample Distribution ========")
print(dfValLabel.value_counts())
print("==========Holdout Sample Distribution ===========")
print(dfHoldOutLabel.value_counts())
print("=================================================")
print("TIME:", time.time() - st)

Applying sum 23.31420588493347
Calculating class_weight 23.596460580825806
IPO 23.62406086921692
Calculating class_weight_aux 60.835963010787964
IPO 64.09716629981995
Weights calculated 106.35609650611877
==================== Data Set ==================================
Holdout Set =>  (270732, 48)
Train Set =>  (1534142, 48)
==================== Data Set ==================================


/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:842: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


==================== Data Set ==================================
Holdout Set =>  (276146, 48)
Train Set =>  (1257996, 48)
==================== Data Set ==================================
==========Train Sample Distribution =============
0.0    985215
1.0     84356
Name: target, dtype: int64
==========Validation Sample Distribution ========
0.0    216204
1.0     18296
Name: target, dtype: int64
==========Holdout Sample Distribution ===========
0.0    249082
1.0     21650
Name: target, dtype: int64
TIME: 168.86105155944824


In [4]:
# df[['target', 'label_imp', 'male', 'class_weight', 'class_weight_aux']][(df['target'] < 0.5) & (df['female'])]
df[['target', 'label_imp', 'male', 'class_weight', 'class_weight_aux']][df['target'] < 0.5]

,target,label_imp,male,class_weight,class_weight_aux
1363214,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
355512,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
1258103,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
915324,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
1042694,0.0,1,False,0.920131,"[0.9201309827722046, 0.9201309827722046, 0.920..."
1166938,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
1124276,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
1202776,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
845305,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."
520603,0.0,0,False,0.080069,"[0.0800690172277954, 0.00010720271886015312, 0..."


### Tokenizer Keras used for tokenization and unique  word count

In [5]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
max_features = 2000000
# tokenizer.fit_on_texts(df['comment_text'].tolist())
tokenizer = text.Tokenizer(num_words = max_features, filters="", lower=True, split=" ")
tokenizer.fit_on_texts(df['comment_text'].tolist() + dfHoldOut['comment_text'].tolist() + dfTest['comment_text'].tolist())

print(len(tokenizer.word_index))
tokenize_dict = tokenizer.word_index

713150


## Embedding Loadup

In [6]:
def load_embeddings_partial(embedings_filepath, tokenize_dict, embedding_size=300):
    def load_embeddings_vec(path):
        def nop(it, *a, **k):
            return it

        def get_coefs(word, *arr):
            return word, np.asarray(arr, dtype='float32')

        tqdm = nop
        with open(path) as f:
            return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

    def load_embeddings_pkl(path):
        with open(path,'rb') as f:
            emb_arr = pickle.load(f)
        return emb_arr
    print("Loading file", embedings_filepath)
    if embedings_filepath.endswith("pkl"):
        embedding_index_ = load_embeddings_pkl(embedings_filepath)
    else:
        embedding_index_ = load_embeddings_vec(embedings_filepath)
    print("Loaded")

    embedding_index = {}
    for key, value in embedding_index_.items():
        embedding_index[key.lower()] = value
    del embedding_index_

    print(len(embedding_index))
    embeddings = np.zeros((len(tokenize_dict) + 1, embedding_size))
    unknown_words = []

    tokens = list(tokenize_dict.keys())
    found = 0
    lookup_index = OrderedDict()
    for i in range(len(tokens)):
        try:
            word = tokens[i]
            lookup_index[word] = i
            embeddings[i] = embedding_index[word]
            found += 1
        except Exception as err:
            unknown_words.append(word)

    del embedding_index
    print("lookup_index:", len(list(lookup_index.keys())))
    
    print("Total Words:", len(tokenize_dict), "Total Unknow Words:", len(unknown_words), "Total Found:", found)
    print("Embeddings shape:", embeddings.shape)

    """ Dictionary which gives index of word in embeddings,
    Eg. lookup_index['word'] will return integer value, and at that index location in embeddings we can find its vector """
    lookup_index_tf = tf.constant(list(lookup_index.keys()))
    table = tf.contrib.lookup.index_table_from_tensor(mapping=lookup_index_tf, default_value=lookup_index["-"])
    gc.collect()
    return embeddings, table, unknown_words

embedding_filepath = "../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl"
embedding_filepath = "../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl"
embedding_filepath = "../input/vector-jigsaw/jigsaw.pkl"
# embedding_filepath = "../input/crawl300d2m/crawl-300d-2M.vec"
# embedding_filepath = "../input/glove-840b-300d/glove.840B.300d.txt"

embeddings, table, unknown_words = load_embeddings_partial(embedding_filepath, tokenize_dict,
                                                           embedding_size=150)
embeddings_tf = tf.placeholder(tf.float32, embeddings.shape, name='embeddings_tf')

print(embeddings.shape)
print(unknown_words[0:100])
gc.collect()

Loading file ../input/vector-jigsaw/jigsaw.pkl
Loaded
792179
lookup_index: 713150
Total Words: 713150 Total Unknow Words: 2138 Total Found: 711012
Embeddings shape: (713151, 150)

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
(713151, 150)
['cannot', 'gonna', 'gotta', 'wanna', 'gimme', '\xa0', 'lemme', '\xa0the', '\u2004\u2004\u2004usa', '\u2009', '\xa0and', '\xa0million', '\xa0a', '\u2009\u2009', '\xa0http', '\u200aand', '\xa0if', '\xa0that', '\u2002', '\u2028', 'of\xa0the', '✨\u2005\u2005', 'repatriate»', 'for\xa0the', '\xa0\xa0', 'tombeau»', '☭\u2004', 'growth\u2028', '\xa0people', '«sir', '»«', '\u2009trillion', '»i', '\xa0of', '\xa0\xa0\xa0\xa0\xa0', '\xa0is', 'that\xa0there', '»theres', '\xa0billion', 'a\xa0', '\u2028\u2028•', 'c\u2009', 'presiden

0

> ### Word2Vec whole vector load

In [7]:
# from gensim.models.keyedvectors import KeyedVectors

# def _load_initial_models(w2v_file):
#     print("Loading W2V Model")
#     model = KeyedVectors.load_word2vec_format(w2v_file, binary=True)
#     dimentions = model.vector_size
#     print("Dimention:", dimentions)
    
#     embeddings = np.zeros((len(model.wv.vocab), dimentions))

#     lookup_index = OrderedDict()
#     for i in range(len(model.wv.vocab)):
#             embedding_vector = model.wv[model.wv.index2word[i]]
#             lookup_index[model.wv.index2word[i]] = i
#             if embedding_vector is not None:
#                     embeddings[i] = embedding_vector

#     """ Dictionary which gives index of word in embeddings,
#     Eg. lookup_index['word'] will return integer value, and at that index location in embeddings we can find its vector """
#     lookup_index_tf = tf.constant(list(lookup_index.keys()))
#     table = tf.contrib.lookup.index_table_from_tensor(mapping=lookup_index_tf, default_value=lookup_index['``'])
#     return embeddings, table

# w2v_file = "../input/word2vec-google/GoogleNews-vectors-negative300.bin"
# w2v_file = "../input/googlenewsvectorsnegative300slim/googlenews-vectors-negative300-slim.bin/GoogleNews-vectors-negative300-SLIM.bin"
# w2v_file = "../input/jigsaw-w2v/jigsaw_model.bin"
# embeddings, table = _load_initial_models(w2v_file)
# embeddings_tf = tf.placeholder(tf.float32, embeddings.shape, name='embeddings_tf')

### Model Creation

In [8]:
n_x  = 150
n_a = 108
n_y = 2
mod = 50
n_y_aux = 6
num_layers = 3
num_memoryunit = 1
MINORFLOAT  = 0.00000001
MODEL_NAME = "BI-LSTM"

In [9]:
# tf.reset_default_graph()
results = {}
G = tf.Graph()
print(G)
with G.as_default():
    counter = tf.Variable(0, name='counter', trainable=False)
    assert counter.graph is G
    
    xav_init = tf.contrib.layers.xavier_initializer
    keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
    learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
    x = tf.placeholder(shape=[None, None, n_x], dtype=tf.float32, name='x')
    y = tf.placeholder(shape=[None], dtype=tf.int32, name='y')
    y_aux = tf.placeholder(shape=[None, n_y_aux], dtype=tf.int32, name='y_aux')
    class_weights = tf.placeholder(shape=[None, 1], dtype=tf.float32)
    class_weights_aux = tf.placeholder(shape=[None, n_y_aux], dtype=tf.float32)

    xt = x
#     xt = tf.transpose(x, [1, 0, 2])
    yt = tf.reshape(y, [-1])  # check this print yt and yb

    """ Multiple LSTM CELLS """
    # Forward direction cell
    lstm_fw_cells = []
    for _ in range(num_layers):
            lstm_fw_cells_m = []
            for _ in range(num_memoryunit):
                
                    lstm_fw_cell_m = tf.nn.rnn_cell.GRUCell(n_a, kernel_initializer=xav_init())
#                     lstm_fw_cell_m = tf.contrib.rnn.BasicLSTMCell(n_a, state_is_tuple=True)
                    lstm_fw_cells_m.append(lstm_fw_cell_m)
            lstm_fw_cells_m = tf.nn.rnn_cell.MultiRNNCell(lstm_fw_cells_m, state_is_tuple=True)
            lstm_fw_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_fw_cells_m, input_keep_prob=keep_prob)
            lstm_fw_cells.append(lstm_fw_cell)

    # Backward direction cell
    lstm_bw_cells = []
    for _ in range(num_layers):
            lstm_bw_cells_m = []
            for _ in range(num_memoryunit):
                    lstm_bw_cell_m = tf.nn.rnn_cell.GRUCell(n_a, kernel_initializer=xav_init())
#                     lstm_bw_cell_m = tf.contrib.rnn.BasicLSTMCell(n_a, state_is_tuple=True)
                    lstm_bw_cells_m.append(lstm_bw_cell_m)
            lstm_bw_cells_m = tf.nn.rnn_cell.MultiRNNCell(lstm_bw_cells_m, state_is_tuple=True)
            lstm_bw_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_bw_cells_m, input_keep_prob=keep_prob)
            lstm_bw_cells.append(lstm_bw_cell)

    """ This is stacking different layers """
    outputs_, _, _= tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw=lstm_fw_cells, cells_bw=lstm_bw_cells, inputs=xt, dtype=tf.float32)
    print("outputs:", outputs_)
    
#     ''' Picking last activation '''
#     outputs = outputs_[:, -1, :]
#     print("outputs:", outputs)
    
    ''' Max and Avg Pool Activation '''
    mean = tf.math.reduce_mean(outputs_, axis=1)
    maximun = tf.math.reduce_max(outputs_, axis=1)
    outputs = tf.concat([mean, maximun], axis=1)
    print("outputs:", outputs)

#     """ START """
#     outputs_, _ = tf.nn.bidirectional_dynamic_rnn(
#         tf.contrib.rnn.BasicLSTMCell(n_a, state_is_tuple=True), tf.contrib.rnn.BasicLSTMCell(n_a, state_is_tuple=True),
#         inputs=xt, dtype=tf.float32
#     )
#     print("outputs", outputs_)
#     outputs_ = tf.concat(outputs_, 2)
#     outputs = outputs_[:, -1, :]

    """ END """
#     #     """ Single LSTM CELL """
#     lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_a, state_is_tuple=True)
#     outputs_, _ = tf.nn.dynamic_rnn(lstm_cell, xt, dtype=tf.float32)

#     ''' CUDA LSTM Bidirectional short version '''
#     lstm = tf.contrib.cudnn_rnn.CudnnLSTM(num_layers=num_layers, num_units=n_a, 
#                                           kernel_initializer = xav_init(),
#                                           dropout= 0.0, direction="bidirectional")
#     outputs_, output_states = lstm(xt)
#     outputs = outputs_[:, -1, :]

    with tf.variable_scope("dnn1") as scope:
        dense = tf.layers.dense(inputs=outputs, units=n_a * 4, kernel_initializer=xav_init(),
                                activation=tf.nn.leaky_relu)

    with tf.variable_scope("dnn2") as scope:
        dense = tf.layers.dense(inputs=dense, units=n_a, kernel_initializer=xav_init(), 
                            activation=tf.nn.leaky_relu)


    logits = tf.layers.dense(inputs=dense,kernel_initializer=xav_init(), 
                             activation=tf.nn.leaky_relu, units=2, name='logits')

    logits_aux = tf.layers.dense(inputs=dense,kernel_initializer=xav_init(), activation=tf.nn.leaky_relu, units=n_y_aux*2, name='logits_aux')

    print(logits)
    print(logits_aux)

    predictions_softmax = tf.nn.softmax(logits)
    predictions_argmax  =  tf.argmax(predictions_softmax, axis=1)

    train_vars   = tf.trainable_variables() 
    l2_loss = tf.add_n([ tf.nn.l2_loss(v) for v in train_vars if 'bias' not in v.name ]) * 0.001
    l2_loss_mean = tf.reduce_mean(l2_loss)
    
#     yt_onehot = tf.one_hot(yt, n_y)
    #     loss_all = tf.nn.weighted_cross_entropy_with_logits(logits=logits, targets=yt_onehot, pos_weight=classes_weights)
#     weight_per_label = tf.linalg.matmul(tf.to_float(yt_onehot), tf.transpose(class_weights)) # shape [batch_size, 2]

    loss_all = tf.math.multiply(class_weights, tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=yt), name='loss_all')
    loss_all_mean = tf.reduce_mean(loss_all)

    logits_auxs = [x for i in range(n_y_aux)]
    loss_all_auxs = [x for i in range(n_y_aux)]
    for i in range(n_y_aux):
        logits_auxs[i] = logits_aux[:, 1:3]
        y_aux_i = y_aux[:, i]
        y_aux_i = tf.reshape(y_aux_i, [-1])
        loss_all_auxs[i] = tf.math.multiply(class_weights_aux[:, i], tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_auxs[i], labels=y_aux_i), 
                                            name='loss_aux' + str(i))
    # loss_train = loss_all + 0.5 * (1 - f1)    
    print(loss_all)
    loss_train = loss_all + l2_loss_mean * 0.005 + loss_all_auxs[0] + loss_all_auxs[1] + loss_all_auxs[2] + loss_all_auxs[3] + loss_all_auxs[4] + loss_all_auxs[5]
    loss = tf.reduce_mean(loss_train)

    tf.summary.scalar('loss_modified', loss)
    tf.summary.scalar('loss', tf.reduce_mean(loss_all))

    #     optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #     tvars = tf.trainable_variables()
    #     grads, _ = tf.clip_by_global_norm(tf.gradients(loss_all, tvars), 10)
    #     train_op = optimizer.apply_gradients(zip(grads, tvars))

    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_train)
    global_step = tf.assign_add(counter, 1, name='global_step')

    cls_sess = tf.Session()
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    cls_sess.run(init_op)
    print(loss.graph)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
outputs: Tensor("stack_bidirectional_rnn/cell_2/concat:0", shape=(?, ?, 216), dtype=float32)
outputs: Tensor("concat:0", shape=(?, 432), dtype=float32)
Instructions for updating:
Use keras.layers.dense instead.
Tensor("logits/LeakyRelu:0", shape=(?, 2), dtype=float32)
Tensor("logits_aux/LeakyRelu:0", shape=(?, 12), dtype=float3

In [10]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & df[label] <= 0.5]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label] > 0.5]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label] > 0.5]
    non_subgroup_negative_examples = df[~df[subgroup] & df[label] <= 0.5]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

def calculate_overall_auc(df, model_name):
    true_labels = df[TARGET_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    print("Overall AUC:", overall_auc)
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

In [11]:
gc.collect()
def parse_function(row):
        row = table.lookup(row)
        output = tf.nn.embedding_lookup(embeddings_tf, row)
        """ See how can we add below features that were their in the original
        pos_tag(sent)
        getWordFeatures(token)
        """
        return output

def getDatasetIterator(ids, texts, labels, auxlabels, class_weight, class_weight_aux, mode="TRAIN", batchsize=2, dimentions=100):
    sentences = tf.data.Dataset.from_tensor_slices(texts)
    sentences = sentences.map(lambda string: tf.string_split([string]).values)
    #sentences = sentences.map(parse_function, num_parallel_calls=4)
    sentences_embed = sentences.map(parse_function)
    if mode in ['TRAIN', 'EVAL']:
        ids =  tf.data.Dataset.from_tensor_slices(ids)
        labels = tf.data.Dataset.from_tensor_slices(labels)        
        auxlabels = tf.data.Dataset.from_tensor_slices(auxlabels)
        class_weight = tf.data.Dataset.from_tensor_slices(class_weight)
        class_weight_aux = tf.data.Dataset.from_tensor_slices(class_weight_aux.tolist())

        dataset = tf.data.Dataset.zip((ids, sentences_embed, labels, auxlabels, class_weight, class_weight_aux))
        if mode == "TRAIN":
            dataset = dataset.shuffle(buffer_size=batchsize*2)

        dataset = dataset.padded_batch(batchsize, padded_shapes=([], [None, dimentions],
                                                                 [], [6], [], [6]))
    elif mode in ['PREDICT']:
        ids =  tf.data.Dataset.from_tensor_slices(ids)
        dataset = tf.data.Dataset.zip((ids, sentences_embed))        
        dataset = dataset.padded_batch(batchsize, padded_shapes=([], [None, dimentions]))
    
    dataset = dataset.prefetch(1)
    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()
    init_op = iterator.initializer
    return init_op, next_element

def execute(df, batchsize=4, mode="TRAIN", max_timestamps=200, comment=""):
    df['comment_text'] = df['comment_text'].apply(lambda x: x.replace("\n", ''))

    init_op_model, next_element_model = getDatasetIterator(df['id'], 
                                            df['comment_text'],
                                            df['target'] if mode in ['TRAIN', 'EVAL'] else None,
                                            df[AUX_COLUMNS] if mode in ['TRAIN', 'EVAL'] else None,
                                            df['class_weight'] if mode in ['TRAIN', 'EVAL'] else None,
                                            df['class_weight_aux'] if mode in ['TRAIN', 'EVAL'] else None,
                                            mode=mode, batchsize=batchsize, dimentions=n_x)

    sess = tf.Session()
    sess.run(tf.tables_initializer())

    print("Iterations:", int(df.shape[0] / batchsize))
    sess.run(init_op_model, feed_dict={embeddings_tf: embeddings})

    eval_loss = []
    train_loss = []
    y_true_all = []
    y_pred_all = []
    predictions = []

    while True:
        st = time.time()
        try:
            batch = sess.run(next_element_model)
            ids = batch[0]
            xb = batch[1]
            if mode in ['TRAIN', 'EVAL']:
                yb = batch[2].astype(np.int)
                yb_aux =  batch[3].astype(np.int)
                class_weight = batch[4]
                class_weight_aux = batch[5]

            if xb.shape[1] > max_timestamps:
                xb = xb[:, 0:max_timestamps, :]

            if mode == "TRAIN":
#                 class_weight = np.absolute(yb - np.mean(yb)) + 0.0001
#                 class_weight_aux = np.absolute(yb_aux - np.mean(yb, axis=0)) + 0.0001
                result = cls_sess.run([train_op, global_step, loss, predictions_argmax, yt, predictions_softmax],
                                       feed_dict = {
                                                x: xb, y: yb, y_aux: yb_aux, 
                                                keep_prob: 0.85, learning_rate: 0.005,
                                                class_weights: class_weight.reshape(-1, 1),
                                                class_weights_aux: class_weight_aux
                                        })
                global_step_, loss_, y_pred, y_true, y_softmax = result[1:6]
                train_loss.append(loss_)
                y_true_all.extend(y_true)
                y_pred_all.extend(y_pred)
                complete_percent = int((len(y_pred_all) / df.shape[0]) * 100 )
                prediction = y_softmax
                prediction = list(zip(ids, prediction[:, 1]))
                predictions.extend(prediction)

                if complete_percent < 3 or complete_percent > 99 or complete_percent % mod == 0 :
                    cmd = ["####", comment, "Progress", str(complete_percent), 
                           "Loss(avg):", str(round(np.mean(train_loss), 2)), 
                           "P:", str(round(metrics.precision_score(y_true_all, y_pred_all), 2)),
                          "R:", str(round(metrics.recall_score(y_true_all, y_pred_all), 2)),
                          "F1:", str(round(metrics.f1_score(y_true_all, y_pred_all), 2)),
                          "Acc:", str(round(metrics.accuracy_score(y_true_all, y_pred_all), 2)),
                          'Time:', str(round(time.time() - st, 2))]
                    cmd = " ".join(cmd)
                    sys.stderr.write('\r\033[2K %s' % (cmd))
                    sys.stderr.flush()
#                 results[global_step_] = {
#                                         'P': round(metrics.precision_score(y_true, y_pred), 2),
#                                         'R': round(metrics.recall_score(y_true, y_pred), 2),
#                                         'F1': round(metrics.f1_score(y_true, y_pred), 2),
#                                         'ACC': round(metrics.accuracy_score(y_true, y_pred), 2),
#                                         'Loss': round(loss_, 3)
#                 }
            elif mode == "EVAL":
#                 class_weight = np.absolute(yb - np.mean(yb)) + 0.0001
#                 class_weight_aux = np.absolute(yb_aux - np.mean(yb, axis=0)) + 0.0001
                result = cls_sess.run([loss, predictions_argmax, yt, predictions_softmax],
                                       feed_dict = { 
                                                x: xb, y: yb, y_aux: yb_aux, keep_prob: 1.0,
                                                class_weights: class_weight.reshape(-1, 1),
                                                class_weights_aux: class_weight_aux
                                       })

                loss_, y_pred, y_true, y_softmax = result[0: 4]
                eval_loss.append(loss_)
                y_true_all.extend(y_true)
                y_pred_all.extend(y_pred)
                prediction = y_softmax
                prediction = list(zip(ids, prediction[:, 1]))
                predictions.extend(prediction)

                complete_percent = int((len(y_pred_all) /  df.shape[0]) * 100)
                if complete_percent < 3 or complete_percent > 99 or complete_percent % mod == 0 :
                    cmd = ["####", comment, "Progress ", str(complete_percent), "Loss(avg):", str(round(np.mean(eval_loss), 2)), 
                           "P:", str(round(metrics.precision_score(y_true_all, y_pred_all), 2)),
                          "R:", str(round(metrics.recall_score(y_true_all, y_pred_all), 2)),
                          "F1:", str(round(metrics.f1_score(y_true_all, y_pred_all), 2)),
                          "Acc:", str(round(metrics.accuracy_score(y_true_all, y_pred_all), 2)),
                          'Time:', str(round(time.time() - st, 2))]
                    cmd = " ".join(cmd)
                    sys.stderr.write('\r\033[2K %s' % (cmd))
                    sys.stderr.flush()
            elif mode == "PREDICT":
                result = cls_sess.run([predictions_softmax], feed_dict = { x: xb, keep_prob: 1.0 })
                prediction = result[0]
                prediction = list(zip(ids, prediction[:, 1]))
                predictions.extend(prediction)
        except tf.errors.OutOfRangeError:
            break
        except Exception as err:
            print(err)

    if mode in ['TRAIN', 'EVAL']:
        resultdf = pd.DataFrame(predictions, columns=['id', MODEL_NAME])
        df = pd.merge(df, resultdf, on='id', how='inner')
        bias_metrics_df = compute_bias_metrics_for_model(df, IDENTITY_COLUMNS, MODEL_NAME, TARGET_COLUMN)
        bias_metrics_df = bias_metrics_df.fillna(0.0)
#         print(bias_metrics_df)    
        auc_score = get_final_metric(bias_metrics_df, calculate_overall_auc(df, MODEL_NAME))
        print("AUC_SCORE:", auc_score)

    gc.collect()
    return predictions

epoch = 3
for i in range(epoch):
    print("Epoch:", i)
    st = time.time()
    execute(df, batchsize=1024, mode="TRAIN", comment="Train")
    print(time.time() - st)
    
    st = time.time()
    execute(dfVal, batchsize=1024, mode="EVAL", comment="Validation Eval")
    print(time.time() - st)

st = time.time()
execute(dfHoldOut, batchsize=1024, mode="EVAL", comment="Holdout Eval")
print(time.time() - st)

Epoch: 0
Iterations: 1228


 #### Train Progress 100 Loss(avg): 0.22 P: 0.74 R: 0.46 F1: 0.57 Acc: 0.94 Time: 4.54

Overall AUC: 0.9327519672897158
AUC_SCORE: 0.9041875554070362
1708.8245990276337
Iterations: 269


 #### Validation Eval Progress  100 Loss(avg): 0.21 P: 0.68 R: 0.6 F1: 0.64 Acc: 0.95 Time: 1.48

Overall AUC: 0.9474375443098653
AUC_SCORE: 0.9218979154894316
233.28076171875
Epoch: 1
Iterations: 1228


...

Overall AUC: 0.9494032883847348
AUC_SCORE: 0.9277324751698244
1901.0030169487
Iterations: 269


 #### Validation Eval Progress  100 Loss(avg): 0.21 P: 0.71 R: 0.58 F1: 0.64 Acc: 0.95 Time: 1.58

Overall AUC: 0.9509249540525702
AUC_SCORE: 0.931262031804692
264.9934277534485
Iterations: 264


 #### Holdout Eval Progress  100 Loss(avg): 0.21 P: 0.71 R: 0.59 F1: 0.64 Acc: 0.95 Time: 1.46

Overall AUC: 0.951588443294957
AUC_SCORE: 0.9317784582637656
274.0971851348877


In [12]:
predictions = execute(dfTest, batchsize=1024, mode="PREDICT", comment="Predictions")
dfResult = pd.DataFrame(predictions, columns=['id', 'prediction'])
dfResult.to_csv("submission.csv", index=False)
dfResult.shape

Iterations: 95


(97320, 2)

## Graphs

In [13]:
# import matplotlib.pyplot as plt
# window = 5
# iterations = sorted(results.keys())
# f1 = pd.Series([results[x]['F1'] for x in iterations]).rolling(window=window).mean()
# p = pd.Series([results[x]['P'] for x in iterations]).rolling(window=window).mean()
# r = pd.Series([results[x]['R'] for x in iterations]).rolling(window=window).mean()
# acc = pd.Series([results[x]['ACC'] for x in iterations]).rolling(window=window).mean()
# loss_vals = pd.Series([results[x]['Loss'] for x in iterations]).rolling(window=window).mean()

# rows = 3
# cols = 2

# def _plot(n, var, color,  xlabel, ylabel):
#     plt.subplot(rows, cols, n)
#     plt.plot(iterations, var, color=color)
#     plt.xlabel(xlabel)
#     plt.ylabel(ylabel)

# _plot(1, f1, 'orange', 'Iterations', 'F1')
# _plot(2, 'blue', p, 'Iterations', 'P')
# _plot(3, 'black', r, 'Iterations', 'R')
# _plot(4, 'green', acc, 'Iterations', 'Acc')
# _plot(5, 'orange', loss_vals, 'Iterations', 'Loss')

# plt.tight_layout()
# plt.show()